# Fractional Cover

## Notebook Summary

This notebook provides a fractional cover product which can be used to represent land cover. 

The basis for this notebook below and the following explanation are from Digital Earth Australia http://geoscienceaustralia.github.io/digitalearthau/notebooks/02_DEA_datasets/Introduction_to_Fractional_Cover.html.

### What is Fractional Cover

Fractional Cover represents the proportion of the land surface that is bare (BS), covered by photosynthetic vegetation (PV), or non-photosynethic vegetation (NPV).

The Fractional Cover product was generated using the spectral unmixing algorithm developed by the Joint Remote Sensing Research Program (JRSRP) which used the spectral signature for each pixel to break it up into three fractions, based on field work that determined the spectral characteristics of these fractions. The fractions were retrieved by inverting multiple linear regression estimates and using synthetic endmembers in a constrained non-negative least squares unmixing model.

The green (PV) fraction includes leaves and grass, the non-photosynthetic fraction (NPV) includes branches, dry grass and dead leaf litter, and the bare soil (BS) fraction includes bare soil or rock.

### Fractional Cover Bands

Bare Soil (bare ground, rock, disturbed) (BS): - Bare Ground (bare soil, rock) percentage; Digital Number 10000 = 100%

Photosythetic Vegetation. (green grass, trees, etc.) (PV): - Photosynthetic Vegetation: Green Vegetation percentage;Digital Number 10000 = 100%

Non-Photosythetic vegetation (litter, dead leaf and branches) (NPV): - Non-Photosynthetic Vegetation (litter, dead leaves andbranches) percentage; Digital Number 10000 = 100%

Unmixing Error (UE): - Unmixing Error. The residual error, defined as the Euclidean Norm of the Residual Vector. High values express less confidence in the fractional components.

This notebook builds on the single L8 image with cloud and water mask provided by Digital Earth Austrlia to provide a median product for a time range. This reduces the influence of cloud and variability associated with water identification. 

The product has also been developed to function for Landsat-7, Landsat-5, Landsat-4 and Sentinel-2. 



### Import required modules

In [1]:
# jupyteronly
# jupyter specific imports
%matplotlib inline
import datacube
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

In [2]:
# generic imports
from pyproj import Proj, transform
from datetime import datetime
from datacube.storage import masking
from datacube_utilities.dc_mosaic import create_max_ndvi_mosaic, create_median_mosaic, create_mosaic, create_mean_mosaic
from datacube_utilities.createAOI import create_lat_lon
from datacube_utilities.clean_mask import landsat_qa_clean_mask
from datacube_utilities.dc_fractional_coverage_classifier import frac_coverage_classify
from datacube_utilities.dc_water_classifier import wofs_classify
from datacube_utilities.interactive_maps import display_map
from datacube_utilities.dc_utilities import write_geotiff_from_xr
import odc.algo
from odc.algo import to_f32, from_float, xr_geomedian

import numpy as np
import xarray as xr
import dask
from dask.distributed import Client

client = Client('dask-scheduler.dask.svc.cluster.local:8786')

client.get_versions(check=True)
client

/opt/conda/envs/cubeenv/lib/python3.6/site-packages/datacube/storage/masking.py:4: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  category=DeprecationWarning)


Client Scheduler: tcp://dask-scheduler.dask.svc.cluster.local:8786 Dashboard: http://dask-scheduler.dask.svc.cluster.local:8787/status,Cluster Workers: 5 Cores: 15 Memory: 100.00 GB


### Initialise

In [ ]:
# jupyteronly
dc = datacube.Datacube(app='ls8-fcscene')
dc

### Set Parameters for cube query.

In [ ]:
# parameters

#set start and end dates for time period of interest
#parameter display_name="Start Date" description='Start of time period window' datatype=date"
time_start = '2019-1-1'
#parameter display_name="End Date" description='End of time period window' datatype=date"
time_end = '2020-1-1'

#parameter display_name="Sensor" description="Satellite to use." datatype="string" options=["SENTINEL_2", "LANDSAT_4", "LANDSAT_5", "LANDSAT_7", "LANDSAT_8"],
platform = "SENTINEL_2"

#set resolution
#parameter display_name="Resolution" description="size of pixes" datatype="int"
res = (30)

#area of interest: load in as wkt
#parameter display_name="Area of Interest" description="The area for which the product is required." datatype="string",
#aoi_wkt = "POLYGON((178.33871143681506 -18.034289086654262,178.5117461047838 -18.034289086654262,178.5117461047838 -18.188308595615112,178.33871143681506 -18.188308595615112,178.33871143681506 -18.034289086654262))"
#aoi_wkt = "POLYGON((178.3744170032213 -17.99805897891774,178.5824705920885 -17.99805897891774,178.5824705920885 -18.177554893316973,178.3744170032213 -18.177554893316973,178.3744170032213 -17.99805897891774))"
#aoi_wkt = "POLYGON((178.39064990349448 -17.985650670944256,178.59183703728354 -17.985650670944256,178.59183703728354 -18.17885964445041,178.39064990349448 -18.17885964445041,178.39064990349448 -17.985650670944256))"
#aoi_wkt = "POLYGON((178.38172351189291 -18.030708248220435,178.5897771007601 -18.030708248220435,178.5897771007601 -18.201691209397307,178.38172351189291 -18.201691209397307,178.38172351189291 -18.030708248220435))"
#aoi_wkt = "POLYGON((178.40786612596605 -18.035691344532218,178.62209952440355 -18.035691344532218,178.62209952440355 -18.19231893178261,178.40786612596605 -18.19231893178261,178.40786612596605 -18.035691344532218))"
#aoi_wkt = "POLYGON((178.3996263798723 -17.95013985022938,178.71411002245043 -17.95013985022938,178.71411002245043 -18.181881454834457,178.3996263798723 -18.181881454834457,178.3996263798723 -17.95013985022938))"
#aoi_wkt = "POLYGON ((177.62557983398438 -17.590848708679893, 177.77372360229492 -17.590848708679893, 177.77372360229492 -17.488875828028657, 177.62557983398438 -17.488875828028657, 177.62557983398438 -17.590848708679893))"
#aoi_wkt = "POLYGON((-179.07671013200047 -17.137047418895328,-178.8363842042661 -17.137047418895328,-178.8363842042661 -17.37508032689541,-179.07671013200047 -17.37508032689541,-179.07671013200047 -17.137047418895328))"
# Larger dataset ~4kx4k
#aoi_wkt = "POLYGON((177.36088769010448 -17.306542777663157,178.51445214322948 -17.306542777663157,178.51445214322948 -18.315892730616866,177.36088769010448 -18.315892730616866,177.36088769010448 -17.306542777663157))"
#this is a constant across FIJI so maybe put it not as a variable but built into query.
aoi_wkt = "POLYGON((178.7032001619705 -17.60847609556609,178.87554818443144 -17.60847609556609,178.87554818443144 -17.802749704393555,178.7032001619705 -17.802749704393555,178.7032001619705 -17.60847609556609))"


#parameter display_name="Coordinate Reference System" description="The EPSG code for the CRS, for Fiji this will be 3460." datatype="string" options=["3460", "3832"],
crs = "3460"

In [ ]:
#sort crs stuff
#output_projection = "EPSG:3460"
#crs = "EPSG:3460"

### Reformat inputs

In [ ]:
#createAOI
lat_extents, lon_extents = create_lat_lon(aoi_wkt)

In [ ]:
# jupyteronly
#render map to check AOI
display_map(latitude = lat_extents, longitude = lon_extents)

In [ ]:
#reprojection of AOI into input CRS and reformat
inProj  = Proj("+init=EPSG:4326")
outProj = Proj("+init=EPSG:"+crs)
min_lat, max_lat = (lat_extents) 
min_lon, max_lon = (lon_extents)
x_A, y_A = transform(inProj, outProj, min_lon, min_lat)
x_B, y_B = transform(inProj, outProj, max_lon, max_lat)
lat_range = (y_A, y_B)
lon_range = (x_A, x_B)
print(lat_range)
print(lon_range)

In [ ]:
allmeasurements = ["green","red","blue","nir","swir1","swir2"]
#water_measurements = ["water_classification"]
water_measurements = ['watermask']
def create_product_measurement(platform):
    if platform  in ["SENTINEL_2"]:
        product = 's2_esa_sr_granule'
        measurements = allmeasurements + ["coastal_aerosol","scene_classification"]
        ###CHANGE WHEN S2 WOFS READY
        water_product = 's2_water_mlclassification'
    elif platform in ["LANDSAT_8"]:    
        measurements = allmeasurements + ["pixel_qa"]
        product = 'ls8_usgs_sr_scene'
        water_product = 'ls8_water_mlclassification'
    elif platform in ["LANDSAT_7"]:    
        measurements = allmeasurements + ["pixel_qa"]
        product = 'ls7_usgs_sr_scene'
        water_product = 'ls7_water_classification'
    elif platform in ["LANDSAT_5"]:    
        measurements = allmeasurements + ["pixel_qa"]
        product = 'ls5_usgs_sr_scene'
        water_product = 'ls5_water_classification'
    elif platform in ["LANDSAT_4"]:    
        measurements = allmeasurements + ["pixel_qa"]
        product = 'ls4_usgs_sr_scene'
        water_product = 'ls4_water_classification'
    else:
        print("invalid platform")
    return product, measurements, water_product

product, measurement, water_product = create_product_measurement(platform)

In [ ]:
#create resolution
resolution = (-res, res)

In [ ]:
dask_chunks = dict(
    x = 1000,
    y = 1000
)

In [ ]:
#format dates
def createDate(inputStart, inputEnd):
    start = datetime.strptime(inputStart, '%Y-%m-%d')
    end = datetime.strptime(inputEnd, '%Y-%m-%d')
    startDates = start.date()
    endDates = end.date()
    time_period = (startDates, endDates)
    return time_period

time_range = createDate(time_start, time_end)
time_range

In [ ]:
query = {
    'longitude': lon_range,
    'latitude': lat_range,
    'output_crs': "EPSG:"+crs,  
    'resolution': resolution,
    'time': time_range,
    'crs': "EPSG:"+crs,
    'dask_chunks': dask_chunks,
}

### Load data based on cube query 

In [ ]:
ds = dc.load(
    platform = platform,
    product = product,
    measurements = measurement,
    **query
)
ds

In [ ]:
#check if laods are valid
def is_dataset_empty(ds:xr.Dataset) -> bool:
    checks_for_empty = [
                        lambda x: len(x.dims) == 0,      #Dataset has no dimensions
                        lambda x: len(x.data_vars) == 0  #Dataset no variables 
                       ]
    for f in checks_for_empty:
         if f(ds) == True:
                return True
    return False

if is_dataset_empty(ds): raise Exception("DataCube Load returned an empty Dataset." +  
                                               "Please check load parameters for Baseline Dataset!")


## Mask cloud

In [16]:
def look_up_clean(platform, ds):
    if platform  in ["SENTINEL_2"]:
        good_quality = (
            (ds.scene_classification != 0) & # mask out NO_DATA
            (ds.scene_classification != 1) & # mask out SATURATED_OR_DEFECTIVE
            #(ds.scene_classification != 2) & # mask out DARK_AREA_PIXELS
            (ds.scene_classification != 3) & # mask out CLOUD_SHADOWS
            (ds.scene_classification != 8) & # mask out CLOUD_MEDIUM_PROBABILITY
            (ds.scene_classification != 9) & # mask out CLOUD_HIGH_PROBABILITY
            (ds.scene_classification != 10)&  # mask out THIN_CIRRUS
            (ds.scene_classification != 11)  # mask out SNOW
        )
    elif platform in ["LANDSAT_8"]:  
        good_quality = (
            (ds.pixel_qa == 322)  | # clear
            (ds.pixel_qa == 386)  |
            (ds.pixel_qa == 834)  |
            (ds.pixel_qa == 898)  |
            (ds.pixel_qa == 1346) |
            (ds.pixel_qa == 324)  | # water
            (ds.pixel_qa == 388)  |
            (ds.pixel_qa == 836)  |
            (ds.pixel_qa == 900)  |
            (ds.pixel_qa == 1348)
        )
    elif platform in ["LANDSAT_7", "LANDSAT_5", "LANDSAT_4"]:    
        good_quality = (
            (ds.pixel_qa == 66)  | # clear
            (ds.pixel_qa == 130)  |
            (ds.pixel_qa == 68)  | # water
            (ds.pixel_qa == 132)  
        )
    else:
        print("invalid platform")
    return good_quality

In [17]:
good_quality = look_up_clean(platform, ds)
good_quality

,Array,Chunk
Bytes,63.52 MB,438.09 kB
Shape,"(145, 717, 611)","(1, 717, 611)"
Count,2175 Tasks,145 Chunks
Type,bool,numpy.ndarray


### Generate geomedian

In [18]:
%%time
xx_clean = ds.where(good_quality)
xx_clean = odc.algo.keep_good_only(xx_clean, where=good_quality)
scale, offset = (1/10_000, 0)  # differs per product, aim for 0-1 values in float32
xx_clean_32 = to_f32(xx_clean, scale=scale, offset=offset)
yy = xr_geomedian(xx_clean_32, 
                  num_threads=1,  # disable internal threading, dask will run several concurrently
                  eps=0.2*scale,  # 1/5 pixel value resolution
                  nocheck=True)   # disable some checks inside geomedian library that use too much ram

land_composite = from_float(yy, 
                dtype='int16', 
                nodata=-9999,
                #nodata = np.nan,
                scale=1/scale, 
                offset=-offset/scale)
land_composite = land_composite.rename({"x":"longitude", "y":"latitude"})
land_composite

CPU times: user 1.66 s, sys: 168 ms, total: 1.83 s
Wall time: 1.97 s


<xarray.Dataset>
Dimensions:               (latitude: 717, longitude: 611)
Coordinates:
  * latitude              (latitude) float64 3.933e+06 3.933e+06 ... 3.911e+06
  * longitude             (longitude) float64 1.995e+06 1.995e+06 ... 2.013e+06
Data variables:
    green                 (latitude, longitude) int16 dask.array<chunksize=(717, 611), meta=np.ndarray>
    red                   (latitude, longitude) int16 dask.array<chunksize=(717, 611), meta=np.ndarray>
    blue                  (latitude, longitude) int16 dask.array<chunksize=(717, 611), meta=np.ndarray>
    nir                   (latitude, longitude) int16 dask.array<chunksize=(717, 611), meta=np.ndarray>
    swir1                 (latitude, longitude) int16 dask.array<chunksize=(717, 611), meta=np.ndarray>
    swir2                 (latitude, longitude) int16 dask.array<chunksize=(717, 611), meta=np.ndarray>
    coastal_aerosol       (latitude, longitude) int16 dask.array<chunksize=(717, 611), meta=np.ndarray>
    scene_classification  (latitude, longitude) int16 dask.array<chunksize=(717, 611), meta=np.ndarray>

### Generate Fractional Cover from Geomedian

In [19]:
def frac_coverage_classify_wrapper(x):
    return frac_coverage_classify(x, no_data=np.nan)

frac_classes = xr.map_blocks(frac_coverage_classify_wrapper, land_composite)
frac_classes

<xarray.Dataset>
Dimensions:    (latitude: 717, longitude: 611)
Coordinates:
  * latitude   (latitude) float64 3.933e+06 3.933e+06 ... 3.911e+06 3.911e+06
  * longitude  (longitude) float64 1.995e+06 1.995e+06 ... 2.013e+06 2.013e+06
Data variables:
    bs         (latitude, longitude) float32 dask.array<chunksize=(717, 611), meta=np.ndarray>
    pv         (latitude, longitude) float32 dask.array<chunksize=(717, 611), meta=np.ndarray>
    npv        (latitude, longitude) float32 dask.array<chunksize=(717, 611), meta=np.ndarray>

## Mask Fractional Cover By Water and Cloud

In [20]:
def maskWater(inputGeomedian):
    water_scenes = dc.load(product=water_product,
                           measurements = ["watermask", "waterprob"],
                           **query)
    #water_classes = water_scenes.where(water_scenes >= 0)
    good_quality_water = (
        (water_scenes.watermask >= 0) & # no data
        (
           (water_scenes.waterprob <= 5) |
            (water_scenes.waterprob >= 95)
        )
        )
    water_classes = water_scenes.where(good_quality_water)
    #create mosaic of water mask
    water_composite_mean = water_classes.watermask.mean(dim='time')
    water_composite_mean = water_composite_mean.rename({"x":"longitude", "y":"latitude"})
    # mask to removeclouds, cloud shadow, and water.
    maskedLand = inputGeomedian.where((inputGeomedian != np.nan) & (water_composite_mean < 1))
    return maskedLand, water_composite_mean
frac_cov_masked, watermaskoutput = maskWater(frac_classes)

In [21]:
# transfer to array for export
frac_cov_output = frac_cov_masked.to_array()

In [ ]:
%%time
frac_cov_out = frac_cov_output.compute()

### Create a plot showing FC components.

In [ ]:
# jupyteronly
#plot the fractional cover bands. 
scene = 0
plt.figure(figsize=(12,8))
gs = gridspec.GridSpec(2,2) # set up a 2 x 2 grid of 4 images for better presentation

ax1=plt.subplot(gs[0,0])
frac_cov_out.pv.plot(cmap='gist_earth_r', vmin = 0, vmax = 100)
ax1.set_title('PV')

ax2=plt.subplot(gs[1,0])
frac_cov_out.bs.plot(cmap='Oranges', vmin = 0, vmax = 100)
ax2.set_title('BS')

ax3=plt.subplot(gs[0,1])
frac_cov_out.npv.plot(cmap='copper', vmin = 0, vmax = 100)
ax3.set_title('NPV')

plt.tight_layout()
plt.show()

In [ ]:
# jupyteronly
# Plot fractional cover as cloud free RGB image 
frac_cov_out[['bs','pv','npv']].to_array().plot.imshow(
    #col='time',
    figsize=(12, 8),
    vmin=0,
    vmax=100
);

### Export

In [ ]:
#Write as Cog
write_cog(geo_im=frac_cov_out,
          fname='fractional_cover_product.tif',
          overwrite=True)

In [ ]:
#export as Geotiff if required
#write_geotiff_from_xr('fractional_cover.tiff', frac_cov_out, crs=output_projection, x_coord = 'longitude', y_coord = 'latitude')

In [ ]:
#naming exports for ESRI to pick up
['fractional_cover_product.tif']

---